In [1]:
# Imports
import requests

In [2]:
# Billing Status
from Helper import StakingRewardsAPIClient

client = StakingRewardsAPIClient()
status = client.get_billing_status()
print(status)

{'plan_name': 'startup yearly v0', 'available_credits': 1499821, 'monthly_quota': 1500000, 'subscribed_at': '2025-11-24T07:32:41Z', 'requests_allowed': True, 'billing_date': '2025-12-24T07:32:41Z'}


In [5]:
# Asset Queries
eth = client.get_assets(symbols=["ETH"], limit=1)
print(eth)

DEBUG - Query: 
        {
          assets(where: {symbols: ["ETH"]}, limit: 3) {
            id
            name
            slug
            description
            symbol
          }
        }
        
{'data': {'assets': [{'id': '5d27aa0de03dd80007d1eb49', 'name': 'Ethereum', 'slug': 'ethereum-2-0', 'description': "the world's largest and most decentralized Layer1 blockchain. The network is used for building dApps, holding assets, transacting and communicating without being controlled by a central authority. The Ethereum vision is to build a digital future on a global scale, that is powerful enough to help all of humanity", 'symbol': 'ETH'}]}}


In [4]:
result = client.get_asset_metrics(
      slug="polkadot",
      metric_keys=["reward_rate", "provider"],
      created_before="2023-06-28",
      metrics_limit=10
  )
print(result)

{'data': {'assets': [{'id': '5d27aa0ee03dd80007d1eb6d', 'slug': 'polkadot', 'logoUrl': 'https://coin-images.coingecko.com/coins/images/12171/large/polkadot.png?1696512008', 'metrics': [{'defaultValue': 14.835948403683611, 'createdAt': '2023-06-27T22:01:16.026146Z'}, {'defaultValue': 14.835916173737743, 'createdAt': '2023-06-27T20:01:07.394307Z'}, {'defaultValue': 14.835696505476331, 'createdAt': '2023-06-27T18:11:07.667837Z'}, {'defaultValue': 14.835516951370295, 'createdAt': '2023-06-27T16:01:13.578876Z'}, {'defaultValue': 14.852641094515928, 'createdAt': '2023-06-27T14:01:58.737205Z'}, {'defaultValue': 14.852593442400858, 'createdAt': '2023-06-27T12:10:17.547877Z'}, {'defaultValue': 14.852589686723764, 'createdAt': '2023-06-27T10:01:22.273959Z'}, {'defaultValue': 14.85250720223731, 'createdAt': '2023-06-27T08:01:23.624704Z'}, {'defaultValue': 14.8524890814507, 'createdAt': '2023-06-27T06:01:55.861665Z'}, {'defaultValue': 14.852486411180484, 'createdAt': '2023-06-27T04:01:27.988426Z'}

In [4]:
providers = client.get_providers(asset_slug="cosmos")
print(providers)

{'data': {'providers': [{'slug': 'p2p-validator', 'rewardOptions': [{'metrics': [{'defaultValue': 19.06351149935919}]}]}, {'slug': 'figment-networks', 'rewardOptions': [{'metrics': [{'defaultValue': 18.818428911935055}]}]}, {'slug': 'kiln', 'rewardOptions': [{'metrics': [{'defaultValue': 19.01100151971126}]}]}, {'slug': 'allnodes', 'rewardOptions': [{'metrics': [{'defaultValue': 18.649477739423034}]}]}, {'slug': 'stake-fish', 'rewardOptions': [{'metrics': [{'defaultValue': 19.683184510055003}]}]}, {'slug': 'hashquark', 'rewardOptions': [{'metrics': [{'defaultValue': 19.65423787482338}]}]}, {'slug': 'chorus-one', 'rewardOptions': [{'metrics': [{'defaultValue': 19.151953968578734}]}]}, {'slug': 'nansen', 'rewardOptions': [{'metrics': [{'defaultValue': 19.65281496138043}]}]}, {'slug': 'stakin', 'rewardOptions': [{'metrics': [{'defaultValue': 19.67435992498762}]}]}, {'slug': 'stakely', 'rewardOptions': [{'metrics': [{'defaultValue': 19.683700132547347}]}]}]}}


In [4]:
# Total market cap
client.get_metrics(limit=1)

{'data': {'metrics': [{'defaultValue': 3183384258474.3066,
    'changeAbsolutes': '{"1y": -124284384191.89453, "7d": 13913124235.17871, "24h": 99242363989.19092, "30d": -758711371721.6655, "90d": -791154185015.4048}',
    'changePercentages': '{"1y": -3.7574617538385904, "7d": 0.43897305404924386, "24h": 3.217827434160872, "30d": -19.246396914119202, "90d": -19.90556126866289}',
    'createdAt': '2025-11-26T22:03:08.928548Z'}]}}

In [4]:
# Find the top providers for a coin
validators = client.get_validators(symbol="ETH", limit=4)
print(validators)

{'data': {'assets': [{'id': '5d27aa0de03dd80007d1eb49', 'name': 'Ethereum', 'slug': 'ethereum-2-0', 'description': "the world's largest and most decentralized Layer1 blockchain. The network is used for building dApps, holding assets, transacting and communicating without being controlled by a central authority. The Ethereum vision is to build a digital future on a global scale, that is powerful enough to help all of humanity", 'symbol': 'ETH', 'metrics': [{'metricKey': 'active_validators', 'label': 'Active Validators', 'defaultValue': 923301}]}]}}


In [3]:
# Find the staked tokens for a asset slug
result = client.get_staked_tokens(asset_slug="cosmos", limit=10)
print(result)

{'data': {'rewardOptions': [{'providers': None, 'validators': [{'address': None, 'status': {'label': 'unknown'}, 'metrics': None}]}, {'providers': [{'slug': 'chorus-one'}], 'validators': [{'address': 'cosmosvaloper15urq2dtp9qce4fyc85m6upwm9xul3049e02707', 'status': {'label': 'active'}, 'metrics': [{'metricKey': 'staked_tokens', 'defaultValue': 5278968.642348851}]}]}, {'providers': [{'slug': 'nansen'}], 'validators': [{'address': 'cosmosvaloper1jlr62guqwrwkdt4m3y00zh2rrsamhjf9num5xr', 'status': {'label': 'active'}, 'metrics': [{'metricKey': 'staked_tokens', 'defaultValue': 4086149.456492351}]}]}, {'providers': [{'slug': 'p2p-validator'}], 'validators': [{'address': 'cosmosvaloper132juzk0gdmwuxvx4phug7m3ymyatxlh9734g4w', 'status': {'label': 'active'}, 'metrics': [{'metricKey': 'staked_tokens', 'defaultValue': 5342077.522632887}]}, {'address': 'wl_node_cosmos_p2p-validator_1', 'status': {'label': 'active'}, 'metrics': [{'metricKey': 'staked_tokens', 'defaultValue': 249258}]}]}, {'provider